In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers -q
!pip install nltk
!pip install pandarallel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=ccf5c02c8eb139d555a6ccd91c3020956b4f9cd8cb7a6a5195ec35f7fb7c650e
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [ ]:
!pip install annoy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Importing Libraries
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()

import _pickle as cPickle
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import psutil

workers = psutil.cpu_count()
# psutil.cpu_count(logical=False)

In [ ]:
import pandas as pd

from datetime import datetime
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModel, BertModel
from sklearn.metrics.pairwise import cosine_similarity


import re
from gensim.models import Word2Vec    #Import gensim Word2Fec
from sklearn.decomposition import PCA #Grab PCA functions
import numpy as np

#Plot helpers
import matplotlib.pyplot as plt

#Enable matplotlib to be interactive (zoom etc)
import ast

from pandarallel import pandarallel

# from pandarallel.utils import progress_bars
# progress_bars.is_notebook_lab = lambda : True

pandarallel.initialize(progress_bar=True, nb_workers=workers, use_memory_fs=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Loading Data and Lexicon


In [ ]:
# df = pd.read_csv(r"/content/drive/MyDrive/Krishnin/Resources/NRC-Modified-Final.csv")
df = pd.read_csv(r"/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity Emb.csv")

# df['embedding'] = df['embedding'].tolist()
df['embedding'] = [ast.literal_eval(i) for i in df['embedding'].values.tolist()]
print(df.head())
print(df.columns)
df = df.dropna()

        word emotion  emotion-intensity-score  \
0   outraged   anger                    0.964   
1  brutality   anger                    0.959   
2     hatred   anger                    0.953   
3    hateful   anger                    0.940   
4  terrorize   anger                    0.939   

                                           embedding  
0  [-0.13173015415668488, 0.1441633105278015, 0.1...  
1  [-0.16712629795074463, 0.1456366330385208, 0.1...  
2  [-0.14905360341072083, 0.11977861076593399, 0....  
3  [-0.16983617842197418, 0.15502475202083588, 0....  
4  [0.04510786011815071, 0.093549445271492, 0.192...  
Index(['word', 'emotion', 'emotion-intensity-score', 'embedding'], dtype='object')


In [ ]:
print(df.groupby('emotion').count())
print(df.groupby('emotion').sum())

              word  emotion-intensity-score  embedding
emotion                                               
anger         1483                     1483       1483
anticipation   864                      864        864
disgust       1094                     1094       1094
fear          1765                     1765       1765
joy           1268                     1268       1268
sadness       1298                     1298       1298
surprise       585                      585        585
trust         1564                     1564       1564
              emotion-intensity-score
emotion                              
anger                         740.397
anticipation                  431.978
disgust                       547.167
fear                          881.033
joy                           635.373
sadness                       650.952
surprise                      292.496
trust                         782.109


<ipython-input-9-13a367767a6f>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print(df.groupby('emotion').sum())


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9921 entries, 0 to 9920
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     9921 non-null   object 
 1   emotion                  9921 non-null   object 
 2   emotion-intensity-score  9921 non-null   float64
 3   embedding                9921 non-null   object 
dtypes: float64(1), object(3)
memory usage: 310.2+ KB


In [ ]:
len(df['embedding'][0])

768

## Import Annoy

In [ ]:
from annoy import AnnoyIndex
import random

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

# super fast, will just mmap the file
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/Aug - 4th week/angular_nrc_embedding_space_V3.ann')
t.load(r'/content/drive/MyDrive/Krishnin/Resources/NRC Emotion Intensity Emb.ann')

True

In [ ]:
t.get_n_items()

9921

# Hateval 2019 Dataset



In [ ]:
dff = pd.read_csv("/content/drive/MyDrive/hateval2019-relabeled-main/hateval2019_en_train_relabeled.csv")

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    8987 non-null   object
 1   label   9000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 140.8+ KB


In [ ]:
dff.head()

,text,label
0,"Hurray, saving us $$$ in so many ways #LockThe...",1
1,Why would young fighting age men be the vast m...,1
2,Illegals Dump their Kids at the border like Ro...,1
3,NY Times: 'Nearly All White' States Pose 'an A...,0
4,Orban in Brussels: European leaders are ignori...,0


In [ ]:
dff.isnull().sum()

text     13
label     0
dtype: int64

In [ ]:
dff.isnull().any(axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
8995    False
8996    False
8997    False
8998    False
8999    False
Length: 9000, dtype: bool

# Mean Pooling

In [ ]:
 dff=dff.dropna(axis=0)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
# Function to generate embeddings

def get_embedings(model,sentences):
  model.to(device)   ##to run on gpu
  sent_len=len(sentences)
  limit=1000
  if sent_len>limit:
    sentence_embeddings=[]
    for i in range(0,sent_len,limit):
      j=i+limit
      if j>sent_len:
        j=sent_len
      #Tokenize sentences   Here tokenize and genrtae sentence embeddings in 10000 chunk
      encoded_input = tokenizer(sentences[i:j], padding=True, truncation=True, return_tensors='pt')
      encoded_input.to(device)
      #Compute token embeddings
      with torch.no_grad():
          model_output = model(**encoded_input)
      #Perform pooling. In this case, mean pooling
      sentence_embeddings1 = mean_pooling(model_output, encoded_input['attention_mask'])
      sentence_embeddings = sentence_embeddings+sentence_embeddings1.tolist()
  else :
      encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
      encoded_input.to(device)
      #Compute token embeddings
      with torch.no_grad():
          model_output = model(**encoded_input)
      #Perform pooling. In this case, mean pooling
      sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
      sentence_embeddings = sentence_embeddings.tolist()
  print(len(sentence_embeddings))
  return sentence_embeddings

In [ ]:
# Defining the model here, we use a model that is fine tuned on News articles
# model_name = "/content/drive/MyDrive/Krishnin/Resources/go_model"
model_name = "vinai/bertweet-base"

# Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
sentences = dff['text'].values.tolist()
print(len(sentences))

8987


In [ ]:
emotion_embeddings = get_embedings(model, sentences)

8987


In [ ]:
len(emotion_embeddings)

8987

In [ ]:
dff['embedding'] = emotion_embeddings

# Emotion Extraction Pipeline

In [ ]:
pandarallel.initialize()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
def get_nearest_neighbours(index):
    tuples = []

    embeding = dff['embedding'][index]
    # sentence_tokens = dff['text'][index].split(' ')


    # for i,row_e in df.iterrows():

    #     dis = cosine_similarity([row_e['embedding']], [embeding])
    #     # print([row_e['tokens'],row_d['tokens'],dis])
    #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

    indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
    for i in range(len(indexes)):
      tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                    #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                     ])
    # print(indexes)
    # print(distances)
    # print(tuples)

    s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
    neaarest_neighbs_words = []
    neaarest_neighbs_embs = []
    neaarest_neighbs_labels = []
    neaarest_neighbs_distance = []
    neaarest_neighbs_intensity_score = []
    for i,m in enumerate(s_tup[::-1]):
        # print(m)
        if(i<50):#getting the nearest 50 neighbours
            neaarest_neighbs_words.append(m[0])
            neaarest_neighbs_embs.append(m[3])
            neaarest_neighbs_labels.append(m[1])
            # distance = m[2].tolist()[0][0]
            distance = m[2]
            neaarest_neighbs_distance.append(distance)
            neaarest_neighbs_intensity_score.append(m[4])

    n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score)

    neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
    normalized_score_dict = neighbour_output[0]

    # for key in normalized_score_dict:
    #   normalized_score_dict[key] = round(normalized_score_dict[key],4)

    normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
    # print(normalized_score_dict)
    return normalized_score_dict

In [ ]:
def calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score):
  score_dict = {
              'anticipation':0,
              'anger':0,
              'fear':0,
              'sadness':0,
              'trust':0,
              'joy':0,
              'surprise':0,
              'disgust':0
              }

  #Scoring Mechanism
  for i in range(0,len(neaarest_neighbs_words)):
    distance = 0
    if neaarest_neighbs_distance[i] == 0 :
      distance = 1
    else :
      distance = (1/neaarest_neighbs_distance[i])

    score = (distance*(neaarest_neighbs_intensity_score[i]))

    score_dict[neaarest_neighbs_labels[i]]=score_dict[neaarest_neighbs_labels[i]]+score


  #Normalising Mechanism
  normalized_score_dict = score_dict.copy()
  # for k in score_dict.keys():
  #   if score_dict[k] ==0:
  #     continue
  #     # del normalized_score_dict[k]
  #   else:
  #     normalized_score_dict[k]
  #     # normalized_score_dict[k] = round((score_dict[k]/score_max),3)

  return normalized_score_dict

In [ ]:
import operator

def dict_to_result(emotion_dict):
  if (all(value == 0 for value in emotion_dict.values())):
    highest_8 = None
  else:
    highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
  return highest_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [ ]:
dff['index'] = dff.index

In [ ]:
dff.tail()

,text,label,embedding,index
8995,I am proud to be a hysterical woman.,0,"[0.06200055778026581, 0.08094975352287292, 0.1...",8995
8996,Hollywood is complicit in the rape and sexual ...,0,"[-0.10346726328134537, 0.11953989416360855, 0....",8996
8997,What a fucking cunt I hate seeing kids getting...,1,"[0.10017873346805573, 0.09013339132070541, 0.0...",8997
8998,Hysterical woman like,0,"[-0.3113396167755127, -0.032951585948467255, 0...",8998
8999,Nearly every woman I know has #meToo in their ...,0,"[-0.012088735587894917, 0.06046191230416298, 0...",8999


In [ ]:
dff['emotion_score_dict'] = dff['index'].parallel_apply(get_nearest_neighbours)

In [ ]:
dff['highest_eight_label'] = dff['emotion_score_dict'].parallel_apply(dict_to_result)

In [ ]:
eight_label = dff['emotion_score_dict'].to_list()

print(eight_label)
dff["anger"]=[i["anger"] for i in eight_label]
dff["anticipation"]=[i["anticipation"] for i in eight_label]
dff["disgust"]=[i["disgust"] for i in eight_label]
dff["fear"]=[i["fear"] for i in eight_label]
dff["joy"]=[i["joy"] for i in eight_label]
dff["sadness"]=[i["sadness"] for i in eight_label]
dff["surprise"]=[i["surprise"] for i in eight_label]
dff["trust"]=[i["trust"] for i in eight_label]

[{'joy': 0.608, 'surprise': 1.014, 'anticipation': 1.674, 'sadness': 2.367, 'fear': 3.419, 'trust': 4.593, 'disgust': 5.344, 'anger': 8.005}, {'joy': 0, 'trust': 2.358, 'anticipation': 3.016, 'surprise': 3.501, 'sadness': 4.746, 'fear': 5.713, 'anger': 6.16, 'disgust': 6.738}, {'joy': 1.874, 'surprise': 2.023, 'anticipation': 2.586, 'trust': 3.489, 'sadness': 3.582, 'fear': 5.739, 'anger': 5.746, 'disgust': 5.883}, {'surprise': 0.316, 'joy': 0.877, 'anticipation': 3.504, 'disgust': 3.533, 'anger': 4.649, 'fear': 4.683, 'sadness': 4.795, 'trust': 5.319}, {'joy': 1.463, 'anticipation': 2.762, 'surprise': 3.018, 'trust': 3.216, 'anger': 3.534, 'sadness': 4.154, 'disgust': 4.517, 'fear': 6.753}, {'surprise': 0.596, 'joy': 2.249, 'anticipation': 2.267, 'sadness': 4.181, 'anger': 4.792, 'disgust': 4.796, 'fear': 6.063, 'trust': 7.271}, {'joy': 0, 'anticipation': 1.786, 'trust': 2.277, 'surprise': 2.57, 'sadness': 5.226, 'disgust': 6.216, 'anger': 6.293, 'fear': 9.008}, {'joy': 0.188, 'surpri

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8987 entries, 0 to 8999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 8987 non-null   object 
 1   label                8987 non-null   int64  
 2   embedding            8987 non-null   object 
 3   index                8987 non-null   int64  
 4   emotion_score_dict   8987 non-null   object 
 5   highest_eight_label  8987 non-null   object 
 6   anger                8987 non-null   float64
 7   anticipation         8987 non-null   float64
 8   disgust              8987 non-null   float64
 9   fear                 8987 non-null   float64
 10  joy                  8987 non-null   float64
 11  sadness              8987 non-null   float64
 12  surprise             8987 non-null   float64
 13  trust                8987 non-null   float64
dtypes: float64(8), int64(2), object(4)
memory usage: 1.0+ MB


In [ ]:
dff.head()

,text,label,embedding,index,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,"Hurray, saving us $$$ in so many ways #LockThe...",1,"[0.08123258501291275, 0.07684978097677231, 0.2...",0,"{'joy': 0.608, 'surprise': 1.014, 'anticipatio...",anger,8.005,1.674,5.344,3.419,0.608,2.367,1.014,4.593
1,Why would young fighting age men be the vast m...,1,"[-0.1006772592663765, -0.03763498738408089, -0...",1,"{'joy': 0, 'trust': 2.358, 'anticipation': 3.0...",disgust,6.160,3.016,6.738,5.713,0.000,4.746,3.501,2.358
2,Illegals Dump their Kids at the border like Ro...,1,"[0.18161311745643616, -0.023667680099606514, 0...",2,"{'joy': 1.874, 'surprise': 2.023, 'anticipatio...",disgust,5.746,2.586,5.883,5.739,1.874,3.582,2.023,3.489
3,NY Times: 'Nearly All White' States Pose 'an A...,0,"[-0.12653376162052155, 0.10920768231153488, -0...",3,"{'surprise': 0.316, 'joy': 0.877, 'anticipatio...",trust,4.649,3.504,3.533,4.683,0.877,4.795,0.316,5.319
4,Orban in Brussels: European leaders are ignori...,0,"[0.006114819552749395, -0.04434633255004883, 0...",4,"{'joy': 1.463, 'anticipation': 2.762, 'surpris...",fear,3.534,2.762,4.517,6.753,1.463,4.154,3.018,3.216


In [ ]:
dff.loc[98,"emotion_score_dict"]

{'surprise': 0,
 'joy': 0.264,
 'anticipation': 2.248,
 'disgust': 3.621,
 'fear': 4.204,
 'sadness': 5.435,
 'anger': 5.757,
 'trust': 5.877}

In [ ]:
out_dff = dff.copy()

In [ ]:
out_dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8987 entries, 0 to 8999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 8987 non-null   object 
 1   label                8987 non-null   int64  
 2   embedding            8987 non-null   object 
 3   index                8987 non-null   int64  
 4   emotion_score_dict   8987 non-null   object 
 5   highest_eight_label  8987 non-null   object 
 6   anger                8987 non-null   float64
 7   anticipation         8987 non-null   float64
 8   disgust              8987 non-null   float64
 9   fear                 8987 non-null   float64
 10  joy                  8987 non-null   float64
 11  sadness              8987 non-null   float64
 12  surprise             8987 non-null   float64
 13  trust                8987 non-null   float64
dtypes: float64(8), int64(2), object(4)
memory usage: 1.0+ MB


In [ ]:
out_dff.head()

,text,label,embedding,index,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,"Hurray, saving us $$$ in so many ways #LockThe...",1,"[0.08123258501291275, 0.07684978097677231, 0.2...",0,"{'joy': 0.608, 'surprise': 1.014, 'anticipatio...",anger,8.005,1.674,5.344,3.419,0.608,2.367,1.014,4.593
1,Why would young fighting age men be the vast m...,1,"[-0.1006772592663765, -0.03763498738408089, -0...",1,"{'joy': 0, 'trust': 2.358, 'anticipation': 3.0...",disgust,6.160,3.016,6.738,5.713,0.000,4.746,3.501,2.358
2,Illegals Dump their Kids at the border like Ro...,1,"[0.18161311745643616, -0.023667680099606514, 0...",2,"{'joy': 1.874, 'surprise': 2.023, 'anticipatio...",disgust,5.746,2.586,5.883,5.739,1.874,3.582,2.023,3.489
3,NY Times: 'Nearly All White' States Pose 'an A...,0,"[-0.12653376162052155, 0.10920768231153488, -0...",3,"{'surprise': 0.316, 'joy': 0.877, 'anticipatio...",trust,4.649,3.504,3.533,4.683,0.877,4.795,0.316,5.319
4,Orban in Brussels: European leaders are ignori...,0,"[0.006114819552749395, -0.04434633255004883, 0...",4,"{'joy': 1.463, 'anticipation': 2.762, 'surpris...",fear,3.534,2.762,4.517,6.753,1.463,4.154,3.018,3.216


In [ ]:
out_dff.drop(['index'], axis = 1, inplace = True)

In [ ]:
out_dff.to_csv(r"/content/drive/MyDrive/Krishnin/Dataset/Hateval/hateval_emotion_scores.csv",index=False)

In [ ]:
dfff = pd.read_csv('/content/drive/MyDrive/Krishnin/Dataset/Hateval/hateval_emotion_scores.csv')

dfff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8987 entries, 0 to 8986
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   text                 8987 non-null   object 
 1   label                8987 non-null   int64  
 2   embedding            8987 non-null   object 
 3   emotion_score_dict   8987 non-null   object 
 4   highest_eight_label  8987 non-null   object 
 5   anger                8987 non-null   float64
 6   anticipation         8987 non-null   float64
 7   disgust              8987 non-null   float64
 8   fear                 8987 non-null   float64
 9   joy                  8987 non-null   float64
 10  sadness              8987 non-null   float64
 11  surprise             8987 non-null   float64
 12  trust                8987 non-null   float64
dtypes: float64(8), int64(1), object(4)
memory usage: 912.9+ KB


In [ ]:
dfff.head()

,text,label,embedding,emotion_score_dict,highest_eight_label,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,"Hurray, saving us $$$ in so many ways #LockThe...",1,"[0.08123258501291275, 0.07684978097677231, 0.2...","{'joy': 0.608, 'surprise': 1.014, 'anticipatio...",anger,8.005,1.674,5.344,3.419,0.608,2.367,1.014,4.593
1,Why would young fighting age men be the vast m...,1,"[-0.1006772592663765, -0.03763498738408089, -0...","{'joy': 0, 'trust': 2.358, 'anticipation': 3.0...",disgust,6.160,3.016,6.738,5.713,0.000,4.746,3.501,2.358
2,Illegals Dump their Kids at the border like Ro...,1,"[0.18161311745643616, -0.023667680099606514, 0...","{'joy': 1.874, 'surprise': 2.023, 'anticipatio...",disgust,5.746,2.586,5.883,5.739,1.874,3.582,2.023,3.489
3,NY Times: 'Nearly All White' States Pose 'an A...,0,"[-0.12653376162052155, 0.10920768231153488, -0...","{'surprise': 0.316, 'joy': 0.877, 'anticipatio...",trust,4.649,3.504,3.533,4.683,0.877,4.795,0.316,5.319
4,Orban in Brussels: European leaders are ignori...,0,"[0.006114819552749395, -0.04434633255004883, 0...","{'joy': 1.463, 'anticipation': 2.762, 'surpris...",fear,3.534,2.762,4.517,6.753,1.463,4.154,3.018,3.216
